conda create --name pytorch_gpu python=3.8
conda activate pytorch_gpu

conda install pytorch torchvision torchaudio cudatoolkit=11.1 -c pytorch -c conda-forge
python -c"import torch; print(torch.cuda.is_available())"

# Binary Classification

In [ ]:
# import necessary libraries 
import torch 
import math 
print(torch.backends.mps.is_available())
print(torch.backends.mps.is_built())

In [ ]:
# Linear regression in torch using tips dataset 
import torch 
import torch.optim as optim
import seaborn as sns 

tips=sns.load_dataset('tips')

# prepare the data 
X=torch.tensor(tips['total_bill'].values, dtype=torch.float32).view(-1, 1) # input data 
y=torch.tensor(tips['tip'].values, dtype=torch.float32).view(-1, 1) # output data

# define the model
model= torch.nn.Linear(1, 1) # create a linear regression model with one input and one output

# define the loss function and optimizer 
criterion = torch.nn.MSELoss() # mean squared error loss function

# define the optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.01) # stochastic gradient descent optimizer with learning rate of 0.01

# train the model 
for epoch in range(100):
    optimizer.zero_grad() # zero the gradients
    output=model(X) # forward pass 
    loss = criterion(output, y) # calculate the loss
    loss.backward() # backward pass
    optimizer.step() # update the weights
    
    print('Final Loss:', loss.item()) # print the final loss after training the model

# MultiClass Classification

In [ ]:
# import necessary libraries 
import pandas as pd 
import torch 
import torch.nn as nn 
import torch.optim as optim 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder

# Load the titanic dataset 
df= sns.load_dataset('titanic').dropna()

# Data preprocessing 
df['sex']= LabelEncoder().fit_transform(df['sex']) # encode sex column as 0 0r 1 
df['alone']= LabelEncoder().fit_transform(df['alone']) # encode alone column as 0 or 1 

# one-hot encode the 'class' column 
enc= OneHotEncoder()
enc.fit(df[['class']])
onehot=enc.transform(df[['class']]).toarray()
df[['class1', 'class2', 'class3']]=onehot

# Features and target 
X= df[['sex', 'age', 'fare', 'alone', 'class1,', 'class2', 'class3']]# select features 
y= df['pclass'] # select target 

# scaling 
scaler= StandardScaler() # create a standard scalar object 
X= scaler.fit_transform(X) # scale the features

# convert to tensor 
X= torch.tensor(X, dtype=torch.float32) # convert features to tensor
y= torch.tensor(y, dtype=torch.long) # convert target to tensor

# split the data
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state=42)

# Network definition 
class Net(nn.Module):
    def__init__(self):
        super(Net, self).__init__()
        self.fc1= nn.Linear(7, 64) # create a fully connected layer with 7 input features and 64 output features
        self.fc2= nn.Linear(64, 64) # create a fully connected layer with 64 input features and 64 output features
        self.fc3= nn.Linear(64, 4) # create a fully connected layer with 64 input features and 4 output features
    
    def forward(self, x):
        x= torch.relu(self.fc1(x)) # apply ReLu activation function to the output of the first fully connected layer 
        x= torch.relu(self.fc2(x)) # apply ReLU activation function to the output of the second fully connected layer 
        x= self.fc3(x) # apply no activation function to the output of the output of the third fully connected layer 
        return x
# Model, Loss, and Optimizer 
model=Net()# create an instance of the Net class
criterion=nn.CrossEntropyLoss() # create a cross-entropy loss function 
optimizer= optim.Adam(model.parameters(), lr=0.01) # create an adam optimizer with learning rate of 0.01

# Training the loop 
for epoch in range(100):
    optimizer.zero_grad() # zero the gradients
    output=model(X) # forward pass 
    loss = criterion(output, y) # calculate the loss
    loss.backward() # backward pass
    optimizer.step() # update the weights

# Evaluate the model 
model.eval() # set the model in evaluation mode
with torch.no_grad():
    output= model(X_test) # forward pass on the test set
    predicted=torch.argmax(output, dim=1) # get the predicted class for each example
    accuracy= (predicted==y_test).float().mean() # calculate the accuracy

    print('Accuracy:', accuracy.item()) # print the accuracy

# predict some examples
model.eval()# set the model to evaluation mode
with torch.no_grad():
    output=model(X_test[:10]) # forward pass on the first 10 examples in the test set 
    predicted=torch.argmax(output, dim=1) # get the predicted class for each example
    print('predictions:', predicted) # print the predictions

# print the actual values 
print('Actual:', y_test[:10])
